- 基于nn.Module，我们可以通过**Sequential，ModuleList和ModuleDict**三种方式定义PyTorch模型。
- 下面我们就来逐个探索这三种模型定义方式。
### sequential（nn.Sequential）
- 当模型的前向计算为简单串联各个层的计算时， Sequential 类可以通过更加简单的方式定义模型。
    - 它可以接收一个子模块的有序字典(OrderedDict) 或者一系列子模块作为参数来逐一添加 Module 的实例，
    - ⽽模型的前向计算就是将这些实例按添加的顺序逐⼀计算。
- 我们结合Sequential和定义方式加以理解：
- **我们先自己实现一遍Seuqential方法**

In [2]:
import torch
from torch import nn

class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): 
            # 如果传入的是一个OrderedDict
            for key, modeule in args[0].items():
                self.add_module(key, module)
                # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:# 如果传入的是一些Module而不是OrderedDict
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
        # self._modules 返回一个OrderedDict,保证会按照成员添加时的顺序遍历
        for module in self._modules.values():
            input = module(input)
        return input

- **下面看直接利用pytorch内置的Sequential方法，简单方便**
1. (Sequential使用)直接排列

In [3]:
import torch.nn as nn
net = nn.Sequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256,10),  #注意这里最后有一个逗号
        )
print(net)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


- (Sequential使用)使用OrderedDict:


In [4]:
import collections
import torch.nn as nn

net2 = nn.Sequential(collections.OrderedDict([
            ('fc1', nn.Linear(784, 256)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(256, 10))
            ]))
print(net2)

Sequential(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


#### summary for Sequential
- 优点：
    - 使用Sequential定义模型的好处在于简单、易读
    - 同时使用Sequential定义的模型不需要再写forward，因为顺序已经定义好了
- 缺点：
    - 但使用Sequential也会使得模型定义丧失灵活性，比如需要在模型中间加入一个外部输入时就不适合用Sequential的方式实现。

- 使用时需根据实际需求加以选择。

### ModuleList
- 对应模块为nn.ModuleList()
- ModuleList接收一个子模块（或层，需属于nn.Module类）的列表作为输入，然后也就可以类似List那样进行append和extend操作。
- 同时，子模块或层的权重也会自动添加到网络中来。


In [5]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10))  # # 类似List的append操作
print(net[-1])   # 类似List的索引访问
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


- 要特别注意的是，nn.ModuleList 并没有定义一个网络，它只是将不同的模块储存在一起。
- **ModuleList中元素的先后顺序并不代表其在网络中的真实位置顺序，需要经过forward函数指定各个层的先后顺序后才算完成了模型的定义**。具体实现时用for循环即可完成：
```
class model(nn.Module):
  def __init__(self, ...):
    self.modulelist = ...
    ...
    
  def forward(self, x):
    for layer in self.modulelist:
      x = layer(x)
    return x
```

### ModuleDict
- 对应模块为nn.ModuleDict()。
- ModuleDict和ModuleList的作用类似(**和ModuleList的区别：只是ModuleDict能够更方便地为神经网络的层添加名称**)。

In [7]:
net = nn.ModuleDict({
    'Linear':nn.Linear(784, 256),
    'act':nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['Linear'])   # 访问
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (Linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


- 三种方法的比较和适用场景
    - Sequential适用于快速验证结果，因为已经明确了要用哪些层，直接写一下就好了，不需要同时写\__init__和forward；

    - ModuleList和ModuleDict在某个完全相同的层需要重复出现多次时，非常方便实现，可以”一行顶多行“；

    - 当我们需要之前层的信息的时候，比如 ResNets 中的 残差计算，当前层的结果需要和之前层中的结果进行融合，一般使用 ModuleList/ModuleDict 比较方便。

## 利用模型块快速搭建复杂网络
- 当模型的深度非常大时候，使用Sequential定义模型结构需要向其中添加几百行代码，使用起来不甚方便。
- 解决方法：
    - 对于大部分模型结构（比如ResNet、DenseNet等），我们仔细观察就会发现，虽然模型有很多层， 但是其中有很多重复出现的结构。
    - 考虑到每一层有其输入和输出，若干层串联成的”模块“也有其输入和输出，
    - 如果我们能将这些重复出现的层定义为一个”模块“，每次只需要向网络中添加对应的模块来构建模型，这样将会极大便利模型构建的过程。
    
### 学习目标
- 利用上一节学到的知识，将简单层构建成具有特定功能的模型块
- 利用模型块构建复杂网络
---------
- UNet是一个复杂且模块重用性比较高的模型（实践）

- 这里的基础部件对应上一节分析的四个模型块，根据功能我们将其命名为：**DoubleConv, Down, Up, OutConv**。下面给出U-Net中模型块的PyTorch 实现：

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class DoubleConv(nn. Module):
    """(convolution => [BN] => ReLU) * 2"""
    
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels():
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size = 3, padding=1,
                     bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1,
                     bias = False),
            nn.batchNorm2d(out_channels),
            nn.ReLU(inplace = True)
        )
        
    def forward(self, x):
        return self.double_conv(x)

In [11]:
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.maxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
        
    def forward(self, x):
        return self.maxpool_conv(x)

In [13]:
class Up(nn.Module):
    """Upscaling then double conv"""
    
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        
        # if bililnear, use the normal convolutions to reduce the number of channels
        if bilinear():
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', 
                            align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels //2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)
            
    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = X2.size()[3] - x1.size()[3]
        
        X1 = F.pad(x1, [diffX //2, diffX - diffX // 2,
                       diffX //2, diffX - diffX // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)
        

In [14]:
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

### 利用模型块组装U-Net
- 使用写好的模型块，可以非常方便地组装U-Net模型。
- 可以看到，**通过模型块的方式实现了代码复用，整个模型结构定义所需的代码总行数明显减少，代码可读性也得到了提升**。

In [16]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet,self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self,x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits